In [ ]:
!pip install tensorflow
!pip install nltk
!pip install flask
!pip install tensorflow

In [ ]:
import random
import re
import os
import numpy as np
import pickle
import json
import nltk

from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
from nltk.stem import WordNetLemmatizer

In [ ]:
#initialize Lemmatizer

lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#intialize files & Load training data

words = []
classes = []
documents = []
ignore_words = ["?", "!", ";", ":"]
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [ ]:
#tokenize words -> break down a sentence; each word forms a token

for intent in intents["intents"]:
  for pattern in intent["patterns"]:

    w = nltk.word_tokenize(pattern)
    words.extend(w)

    #add documents
    documents.append((w, intent["tag"]))

    #add classes to main class list
    if intent["tag"] not in classes:
      classes.append(intent["tag"])

In [ ]:
#lemmatize words -> reducing words to their base form

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))


200 documents
55 classes ['AI', 'abbr', 'artificial', 'bend', 'body', 'bot1', 'breathe', 'business', 'chatbot', 'chatterbox', 'clone', 'comp', 'computer', 'control', 'cramped', 'date', 'death', 'do', 'events', 'fav', 'fight', 'goodbye', 'greetings', 'hardware', 'hobby', 'idea', 'imortal', 'lang', 'laugh', 'lie', 'machine', 'malfunction', 'motormouth', 'move', 'name', 'name1', 'need', 'noanswer', 'os', 'program', 'programming', 'ratchet', 'robotics', 'robots', 'robotss', 'sapient', 'sense', 'sentiment', 'shoe', 'sound', 'stupid', 'thanks', 'usage', 'who', 'wt']
126 unique lemmatized words ["'m", "'s", ',', 'a', 'ai', 'all', 'allowed', 'am', 'an', 'are', 'artificial', 'awesome', 'be', 'being', 'bend', 'body', 'bot', 'breathe', 'business', 'bye', 'can', 'chat', 'chatterbox', 'clone', 'coffee', 'computer', 'control', 'cramped', 'data', 'date', 'die', 'do', 'entity', 'event', 'favorite', 'favour', 'fight', 'for', 'good', 'great', 'hardware', 'haroo', 'hello', 'help', 'helpful', 'helping', '

In [ ]:
#create a Bag of Words -> train model w/ words to recognize patterns

training = []
output_empty = [0] * len(classes)

for doc in documents:
  #initialise BoW
  bag = []

  #list of tokenized words for the pattern
  pattern_words = doc[0]

  #lemmatize each word
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

  #create BoW -> indicate 1 if word is found in current pattern
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    #0 for each tag and 1 for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

  #shuffle features & convert into np.array
  random.shuffle(training)

  #separate BoW represetations & output labels
  train_x = [item[0] for item in training]
  train_y = [item[1] for item in training]

  #convert to NumPy arrays
  train_x = np.array(train_x)
  train_y = np.array(train_y)
  print("Training data created")


Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training data created
Training d

In [ ]:
#create a model w/ 3 layers;
#layer 1 - 128 neurons
#layer 2 - 64 neurons
#layer 3 - no. of neurons = no. of intents to predict output intent w/ softmax

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16256     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 55)                3575      
                                                                 
Total params: 28087 (109.71 KB)
Trainable params: 28087 (109.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#compile model

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [ ]:
#fitting & saving the model

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model_v3.h5", hist)
print("Model created")

Epoch 1/200
6318/6318 [==============================] - 32s 5ms/step - loss: 0.0719 - accuracy: 0.9841
Epoch 2/200
6318/6318 [==============================] - 22s 3ms/step - loss: 0.0798 - accuracy: 0.9817
Epoch 3/200
6318/6318 [==============================] - 21s 3ms/step - loss: 0.0726 - accuracy: 0.9846
Epoch 4/200
6318/6318 [==============================] - 22s 3ms/step - loss: 0.1133 - accuracy: 0.9779
Epoch 5/200
6318/6318 [==============================] - 26s 4ms/step - loss: 0.1614 - accuracy: 0.9716
Epoch 6/200
6318/6318 [==============================] - 23s 4ms/step - loss: 0.2156 - accuracy: 0.9663
Epoch 7/200
6318/6318 [==============================] - 22s 3ms/step - loss: 0.4993 - accuracy: 0.9337
Epoch 8/200
6318/6318 [==============================] - 22s 4ms/step - loss: 0.8628 - accuracy: 0.8988
Epoch 9/200
6318/6318 [==============================] - 23s 4ms/step - loss: 1.0743 - accuracy: 0.8247
Epoch 10/200
6318/6318 [==============================] - 23s 4m

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

In [ ]:
#connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

#fitting & saving the model ->> round 2

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('/content/drive/My Drive/ColabModels/chatbot_v3.h5', hist)
print("Model created")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/200
5040/5040 [==============================] - 20s 3ms/step - loss: 0.7701 - accuracy: 0.7854
Epoch 2/200
5040/5040 [==============================] - 16s 3ms/step - loss: 0.2129 - accuracy: 0.9286
Epoch 3/200
5040/5040 [==============================] - 17s 3ms/step - loss: 0.1624 - accuracy: 0.9433
Epoch 4/200
5040/5040 [==============================] - 16s 3ms/step - loss: 0.1593 - accuracy: 0.9465
Epoch 5/200
5040/5040 [==============================] - 16s 3ms/step - loss: 0.1633 - accuracy: 0.9468
Epoch 6/200
5040/5040 [==============================] - 16s 3ms/step - loss: 0.1807 - accuracy: 0.9419
Epoch 7/200
5040/5040 [==============================] - 17s 3ms/step - loss: 0.2014 - accuracy: 0.9392
Epoch 8/200
5040/5040 [==============================] - 18s 4ms/step - loss: 0.2089 - accuracy: 0.9385
Epoch 9/200
5040/5040 [================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16256     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 55)                3575      
                                                                 
Total params: 28087 (109.71 KB)
Trainable params: 28087 (109.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
